In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-extracted-dataset/News_category_extract_dataset.csv


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [12]:
import re
import nltk
from nltk.corpus import stopwords
from keras.utils import to_categorical
from sklearn import preprocessing

In [4]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [5]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [6]:
data = pd.read_csv('../input/news-extracted-dataset/News_category_extract_dataset.csv')
data.drop(columns=['Unnamed: 0'],inplace=True)
data.shape

(70257, 4)

In [7]:
from keras.preprocessing.text import Tokenizer
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 200
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['clean_text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 43059 unique tokens.


In [8]:
from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(data['clean_text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (70257, 200)


In [10]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labels=data['category'].values
labels_clf=le.fit_transform(labels)
classes=le.classes_

In [13]:
cat_labels=labels_clf
cat_labels = to_categorical(cat_labels)

In [14]:
from sklearn.model_selection import train_test_split
Y=cat_labels
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
print(X_test)

(49179, 200) (49179, 4)
(21078, 200) (21078, 4)
[[   0    0    0 ...   94  727  232]
 [   0    0    0 ...  174   18  212]
 [   0    0    0 ...  402  505 1275]
 ...
 [   0    0    0 ... 5098  121 7858]
 [   0    0    0 ... 4497 1108 1736]
 [   0    0    0 ...  130 6186  209]]


In [15]:
vocab_size = 50000
# Max number of words in each complaint.
maxlen = 200
# This is fixed.

print(len(X_train), "Training sequences")
print(len(X_test), "Validation sequences")

49179 Training sequences
21078 Validation sequences


In [23]:
embed_dim = 100  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(4, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [24]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
token_and_position_embedding (None, 200, 100)          5020000   
_________________________________________________________________
transformer_block_2 (Transfo (None, 200, 100)          87632     
_________________________________________________________________
global_average_pooling1d_2 ( (None, 100)               0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 20)                2020      
_________________________________________________________________
dropout_11 (Dropout)         (None, 20)                0   

In [25]:
from keras.callbacks import EarlyStopping
epochs = 2
batch_size = 64
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/2
615/615 [==============================] - 233s 377ms/step - loss: 1.0529 - accuracy: 0.5334 - val_loss: 0.3484 - val_accuracy: 0.8863
Epoch 2/2
615/615 [==============================] - 237s 385ms/step - loss: 0.2517 - accuracy: 0.9217 - val_loss: 0.3038 - val_accuracy: 0.8996
